In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import matplotlib.colors as mcolors
from scipy.stats import kde
import matplotlib as mpl
import sys
from contextlib import redirect_stdout


In [2]:
#old function for original output
def plot_output(file, threshold,ft = True):
    
    files = glob.glob(file)    
    data = pd.DataFrame()
    
    for f in files:
        csv = pd.read_csv(f, names = ["Alive", "Exposed", "Time"])
        data = pd.concat([data,csv],axis=0)
        
    print("Proportion of runs with ASF at end: {}%".format(100*len(data[data.Time == -1])/len(data)))
    print("Proportion of runs with die out before threshold: {}%".format(100*len(data[data.Exposed <= threshold])/len(data)))
    
    if ft:
        output  = data[(data.Exposed > threshold) & (data.Time != -1)]
    else:
         output  = data[(data.Exposed > threshold)]
    return output

In [3]:
def read_output(path, pops = 1):
    
    c_name = ["Alive", "Exposed", "Enter", "Dieout"]
    c_names = []
    
    for i in range(1,pops+1):
        newnames = [s + "_{}".format(i) for s in c_name]
        c_names.extend(newnames)
    
    file_path = path +"/*.csv"
    
    files = glob.glob(file_path)    
    data = pd.DataFrame()
    
    for f in files:
        csv = pd.read_csv(f, names = c_names)
        data = pd.concat([data,csv],axis=0)
    return data

In [4]:
def summary_stat(data, threshold, save=False):
    if save:
        with open(path+'/pop_summary.txt', 'w') as f:
            with redirect_stdout(f):
                 print_stats(data,threshold)
    else:
        print_stats(data,threshold)

In [5]:
def print_stats(data,threshold):

    ns = 4 #number of stats
    nr, nc = data.shape #number of runs
    pops = nc//ns

    #We first want to filter out the fast dieouts in seeded population but will keep IF spreads to other population
    data_fast = data[(data.iloc[:,1] > threshold) | (np.sum(data.iloc[:,ns+1::ns],axis=1) != 0)]
    nrf = data_fast.shape[0]
    print("---------------------------------------------------")
    print()
    print("Population 1")
    print("Proportion of runs with fast ASF die out: {}%".format(np.round(100-100*nrf/nr,4)))
    #print("Note these runs are filtered out from results")

    #We now have out filtered data!, lets start looking at some stats
    #return(data_fast)
    for i in range(1,pops+1):

        data_population = data_fast.iloc[:,ns*(i-1):ns*i].copy() #selecting relevent data

        if i == 1: #seeded population

            data_population = data_population[data_population.iloc[:,3] > 0] # removing endemic data
            number_endemic = data_population.shape[0]
            print("Proportion of runs with ASF at end: {}%".format(np.round(100-100*number_endemic/nrf,4)))
            mean_stats(data_population,i)

        else: #non-seeded populations

            data_population = data_population[data_population.iloc[:,2] != -2]
            number_asf = data_population.shape[0]

            print()
            print("Population {}".format(i))
            print("Proportion of runs where population gets ASF: {}%".format(np.round(100*number_asf/nrf,2)))

            data_population_s = data_population[data_population.iloc[:,1] > threshold] #ASF spreads
            number_spread = data_population_s.shape[0]

            print("Proportion of runs with fast ASF die out: {}%".format(np.round(100-100*number_spread/number_asf,2)))


            data_population_e = data_population_s[data_population_s.iloc[:,3] > 0] #number that are endemic at end
            number_not_endemic = data_population_e.shape[0]
            print("Proportion of runs with ASF at end: {}%".format(np.round(100-100*number_not_endemic/number_spread,4)))
            mean_stats(data_population_e,i)


In [6]:
def mean_stats(data,i):
    mean_data = np.mean(data)
    print()
    print("Filtered Means: ")
    print("Mean groups exposed: ", round(mean_data[1],2))
    print("Mean groups alive: ", round(mean_data[0],2))
    if i == 1:
        print("Mean die out (days): ", round(mean_data[3],2))
    else:
        print("Mean entry (day): ", round(mean_data[2],2))
        print("Mean days till dieout: ", round(mean_data[3]-mean_data[2],2))
    print()
    print("---------------------------------------------------")

In [7]:
path = "/home/callum/Desktop/ACT_3_Low/"
data = read_output(path,3) #reading in data

In [8]:
summary_stat(data,10)

---------------------------------------------------

Population 1
Proportion of runs with fast ASF die out: 16.2%
Proportion of runs with ASF at end: 0.0119%

Filtered Means: 
Mean groups exposed:  40.21
Mean groups alive:  32.17
Mean die out (days):  421.82

---------------------------------------------------

Population 2
Proportion of runs where population gets ASF: 36.42%
Proportion of runs with fast ASF die out: 16.51%
Proportion of runs with ASF at end: 0.0392%

Filtered Means: 
Mean groups exposed:  40.26
Mean groups alive:  32.05
Mean entry (day):  178.86
Mean days till dieout:  412.86

---------------------------------------------------

Population 3
Proportion of runs where population gets ASF: 9.59%
Proportion of runs with fast ASF die out: 12.06%
Proportion of runs with ASF at end: 4.1018%

Filtered Means: 
Mean groups exposed:  37.37
Mean groups alive:  33.14
Mean entry (day):  346.08
Mean days till dieout:  426.86

---------------------------------------------------


In [ ]:
xedges = np.linspace(0,100,101)
yedges = np.linspace(0,100,101)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(6,6))

min_exp = 0
vm=250
data = ACT
ax0 = axs.flat[0]
hist, xedges, yedges = np.histogram2d(data.Exposed, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Exposed, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Exposed, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,100])
ax0.set_title("ACT")
ax0.set_ylabel("Alive (%)")

data = NSW
ax0 = axs.flat[1]
hist, xedges, yedges = np.histogram2d(data.Exposed, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Exposed, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Exposed, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,100])
ax0.set_title("NSW")

data = NT_ED
ax0 = axs.flat[2]
hist, xedges, yedges = np.histogram2d(data.Exposed, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Exposed, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Exposed, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,100])
ax0.set_title("NT Early Dry")
ax0.set_ylabel("Alive (%)")
ax0.set_xlabel("Exposed (%)")

data = NT_LD
ax0 = axs.flat[3]
hist, xedges, yedges = np.histogram2d(data.Exposed, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Exposed, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Exposed, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,100])
ax0.set_title("NT Late Dry")
ax0.set_xlabel("Exposed (%)")


fig.colorbar(sc, ax=axs, label="Counts", fraction=0.33)
fig.suptitle("Effect of location\non ASF dynamics",y=1.08, fontsize=14)
fig.savefig("/home/callum/Results/plots/lethality.png", dpi = 400, bbox_inches='tight')


In [ ]:


xedges = np.linspace(0,365*3,365*3//7)
yedges = np.linspace(0,100,101)

fig, axs = plt.subplots(2, 2, constrained_layout=True, figsize=(6,6))

min_exp = 0
vm =150
data = ACT
ax0 = axs.flat[0]
hist, xedges, yedges = np.histogram2d(data.Time, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Time, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Time, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,365*3])
ax0.set_title("ACT")
ax0.set_ylabel("Alive (%)")

data = NSW
ax0 = axs.flat[1]
hist, xedges, yedges = np.histogram2d(data.Time, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Time, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Time, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,365*3])
ax0.set_title("NSW")

data = NT_ED
ax0 = axs.flat[2]
hist, xedges, yedges = np.histogram2d(data.Time, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Time, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Time, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,365*3])
ax0.set_title("NT Early Dry")
ax0.set_ylabel("Alive (%)")
ax0.set_xlabel("Time (days)")

data = NT_LD
ax0 = axs.flat[3]
hist, xedges, yedges = np.histogram2d(data.Time, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Time, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = ax0.scatter(data.Time, data.Alive, c=c, s=10,vmin=0, vmax=vm, cmap = plt.cm.get_cmap('cividis'))
ax0.set_ylim([min_exp,100])
ax0.set_xlim([min_exp,365*3])
ax0.set_title("NT Late Dry")
ax0.set_xlabel("Time (days)")
ax0.set_ylabel("Alive (%)")


fig.colorbar(sc, ax=axs, label="Counts", fraction=0.33)
fig.suptitle("Effect of Location\non ASF dynamics",y=1.08,fontsize=14)
fig.savefig("/home/callum/Results/plots/lethality_time.png", dpi = 400, bbox_inches='tight')



In [ ]:
data = ACTNT_LD
hist, xedges, yedges = np.histogram2d(data.Time, data.Alive, (xedges, yedges))
xidx = np.clip(np.digitize(data.Time, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(data.Alive, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
sc = plt.scatter(data.Time, data.Alive, c=c, s=10,vmin=0, vmax=80)# cmap = plt.cm.get_cmap('OrRd'))
plt.ylim([min_exp,100])
plt.xlim([min_exp,365*3])
plt.title("NT Late Dry")
plt.xlabel("Time (days)")
plt.ylabel("Alive (%)")

In [ ]:
data = np.copy(ACT.Time)
# Creating plot
fig = plt.figure(figsize =(10, 7))
time_bins = [0, 100, 200, 300,400, 500, 600, 700,800, 900, 1000, 1100,1200]
plt.hist(data, bins = time_bins)

In [ ]:
data = np.copy(NSW.Time)
data[data == 99999] = 1150
# Creating plot
fig = plt.figure(figsize =(10, 7))
time_bins = [0, 100, 200, 300,400, 500, 600, 700,800, 900, 1000, 1100,1200]
plt.hist(data, bins = time_bins)

In [ ]:
data = np.copy(ACTNT_LD.Time)
data[data == 99999] = 1150
# Creating plot
fig = plt.figure(figsize =(10, 7))
time_bins = [0, 100, 200, 300,400, 500, 600, 700,800, 900, 1000, 1100,1200]
plt.hist(data, bins = time_bins)
 

In [ ]:
print_means(ACT)

In [ ]:
print_means(NSW)

In [ ]:
print_means(NT_ED)

In [ ]:
print_means(NT_LD)

In [ ]:
print_means(ACTNT_LD)

In [ ]:
def print_means(Data):
    
    print("----------------------------------------")
    print("Exposed (%): ", round(np.mean(Data["Exposed"]),3))
    print("Alive (%): ", round(np.mean(Data["Alive"]),3))
    print("Die out (days): ", round(np.mean(Data["Time"]),3))
    print("----------------------------------------")


In [ ]:
min(frequency.Time)

In [ ]:
max(frequency.Time)

In [ ]:
min(density.Time)

In [ ]:
max(density.Time)

In [ ]:
e

In [ ]:
100*2.718**(5*0.00665)
